In [1]:
pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip show spacy

Name: spacy
Version: 3.5.1
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: /usr/local/lib/python3.9/dist-packages
Requires: catalogue, cymem, jinja2, langcodes, murmurhash, numpy, packaging, pathy, preshed, pydantic, requests, setuptools, smart-open, spacy-legacy, spacy-loggers, srsly, thinc, tqdm, typer, wasabi
Required-by: en-core-web-sm, fastai


In [3]:
import pandas as pd
import numpy as np

import spacy
# Load English language model
nlp = spacy.load('en_core_web_sm')

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
# Define custom entity labels
LABEL_NAME = 'NAME'
LABEL_ROLE_NAME = 'ROLE_NAME'
LABEL_JOB_TITLE = 'JOB_TITLE'
LABEL_LOCATION = 'LOCATION'
LABEL_PROJECT_NAME = 'PROJECT_NAME'
LABEL_ACRONYM = 'ACRONYM'

In [5]:
## *** V2 *** (Fine Tuning)
# Define function to extract specified entities from text
def extract_entities(text):
  # Create doc object
  doc = nlp(text)
  # Initialize empty list to store extracted entities
  entities = []

  # Extract named entities using doc.ents
  for ent in doc.ents:
      # Extract entity label and text
      label = ent.label_
      entity = ent.text
      
      # Add entity to list if it is a name, role name, job title, or location
      if label == 'PERSON':
          label = LABEL_NAME
          entities.append((entity, label))
      elif label == 'ORG':
          label = LABEL_ROLE_NAME
          entities.append((entity, label))
      elif label == 'JOB':
          label = LABEL_JOB_TITLE
          entities.append((entity, label))
      elif label == 'GPE':
          label = LABEL_LOCATION
          entities.append((entity, label))

  # Extract noun chunks using doc.noun_chunks
  for chunk in doc.noun_chunks:
      # Check if chunk is a proper noun
      if chunk.root.pos_ == 'PROPN':
          # Extract entity label and text
          label = LABEL_ROLE_NAME
          entity = chunk.text
          entities.append((entity, label))

  # Extract verb conjugations using doc.token.pos_
  for token in doc:
      # Check if token is a verb conjugation
      if token.pos_ == 'VERB':
          # Extract entity label and text
          label = LABEL_ROLE_NAME
          entity = token.text
          entities.append((entity, label))

  # Return list of extracted entities
  return entities

In [6]:
text = "John is the project manager for the XYZ project at the San Francisco location. His job title is Senior Manager."
text = "Sarah is the HR manager at the New York location. Her role is to recruit new employees and manage employee benefits."
entities = extract_entities(text)
print(entities)

[('Sarah', 'NAME'), ('New York', 'LOCATION'), ('Sarah', 'ROLE_NAME'), ('recruit', 'ROLE_NAME'), ('manage', 'ROLE_NAME')]


# **V2 - More complex**


In [7]:
## sample text
text = [
"John is the project manager for the XYZ project at the San Francisco location. His job title is Senior Manager.",
"Sarah is the HR manager at the New York location. Her role is to recruit new employees and manage employee benefits.",
"The ABC project is a collaboration between the San Francisco and New York locations. The project acronym is AB-C.",
"The XYZ project is led by Jane, who is a Senior Director. The project is based in London.",
"The ABC project team includes Tom, who is a Data Scientist, and Emily, who is a Software Engineer."
]

In [8]:
## import pandas as pd
# Create data frame with sample text ** ## To be reaplaced with df = pd.read('file.xlsx')
df = pd.DataFrame({'Text': text},index=[0, 1, 2, 3, 4])
df.Text

0    John is the project manager for the XYZ projec...
1    Sarah is the HR manager at the New York locati...
2    The ABC project is a collaboration between the...
3    The XYZ project is led by Jane, who is a Senio...
4    The ABC project team includes Tom, who is a Da...
Name: Text, dtype: object

In [9]:
# Initialize empty list to store extracted entities
entities = []
# Iterate through rows of data frame
for index, row in df.iterrows():
  # Extract text from 'Text' column
  text = row['Text']
  # Pass text to custom NER model function
  extracted_entities = extract_entities(text)
  # Append extracted entities to list and assign labels
  for entity, label in extracted_entities:
    entities.append({'Text': text, 'Entity': entity, 'Label': label})

In [10]:
# Create new data frame using extracted entities and labels
result_df = pd.DataFrame(entities, columns=['Text', 'Entity', 'Label'])
# print(result_df)

In [11]:
result_df

,Text,Entity,Label
0,John is the project manager for the XYZ projec...,John,NAME
1,John is the project manager for the XYZ projec...,XYZ,ROLE_NAME
2,John is the project manager for the XYZ projec...,San Francisco,LOCATION
3,John is the project manager for the XYZ projec...,John,ROLE_NAME
4,John is the project manager for the XYZ projec...,Senior Manager,ROLE_NAME
5,Sarah is the HR manager at the New York locati...,Sarah,NAME
6,Sarah is the HR manager at the New York locati...,New York,LOCATION
7,Sarah is the HR manager at the New York locati...,Sarah,ROLE_NAME
8,Sarah is the HR manager at the New York locati...,recruit,ROLE_NAME
9,Sarah is the HR manager at the New York locati...,manage,ROLE_NAME
